What i want to do: 
- read in postcode dataset
- loop through the code point files 
- join postcodes in each file 
- make sure these are saved and each time loop through a file it doesn't delete earlier merges. 

In [1]:
# Set libraries
import pandas as pd
import glob
import os
import numpy as np
import datetime

In [2]:
# Read in postcode dataset
SWW_postcode = pd.read_csv(r'/Users/annalisasheehan/Documents/Imperial-Diabetes/postcodes/SWW postcodes.csv')

In [3]:
SWW_postcode = SWW_postcode.set_index('postcode')

In [4]:
SWW_postcode

,date
postcode,
EX12TW,21/08/2014
BH25 5LA,24/02/2017
DT7 3AD,05/05/2011
DT7 3BB,16/01/2013
DT7 3BH,21/10/2011
DT7 3BJ,08/07/2016
DT7 3BP,13/02/2014
DT7 3DU,22/09/2011
DT7 3EZ,06/09/2013


In [5]:
files = glob.glob(r'/Users/annalisasheehan/Documents/Imperial-Diabetes/postcodes/codepo_gb/Data/CSV/*')

In [6]:
files

['/Users/annalisasheehan/Documents/Imperial-Diabetes/postcodes/codepo_gb/Data/CSV/ab.csv',
 '/Users/annalisasheehan/Documents/Imperial-Diabetes/postcodes/codepo_gb/Data/CSV/al.csv',
 '/Users/annalisasheehan/Documents/Imperial-Diabetes/postcodes/codepo_gb/Data/CSV/b.csv',
 '/Users/annalisasheehan/Documents/Imperial-Diabetes/postcodes/codepo_gb/Data/CSV/ba.csv',
 '/Users/annalisasheehan/Documents/Imperial-Diabetes/postcodes/codepo_gb/Data/CSV/bb.csv',
 '/Users/annalisasheehan/Documents/Imperial-Diabetes/postcodes/codepo_gb/Data/CSV/bd.csv',
 '/Users/annalisasheehan/Documents/Imperial-Diabetes/postcodes/codepo_gb/Data/CSV/bh.csv',
 '/Users/annalisasheehan/Documents/Imperial-Diabetes/postcodes/codepo_gb/Data/CSV/bl.csv',
 '/Users/annalisasheehan/Documents/Imperial-Diabetes/postcodes/codepo_gb/Data/CSV/bn.csv',
 '/Users/annalisasheehan/Documents/Imperial-Diabetes/postcodes/codepo_gb/Data/CSV/br.csv',
 '/Users/annalisasheehan/Documents/Imperial-Diabetes/postcodes/codepo_gb/Data/CSV/bs.csv',


In [7]:
labels = pd.read_csv(r'/Users/annalisasheehan/Documents/Imperial-Diabetes/postcodes/codepo_gb/Doc/Code-Point_Open_Column_Headers.csv', header = None)

In [8]:
labels

,0,1,2,3,4,5,6,7,8,9
0,Postcode,Positional_quality_indicator,Eastings,Northings,Country_code,NHS_regional_HA_code,NHS_HA_code,Admin_county_code,Admin_district_code,Admin_ward_code


In [9]:
labels = labels.set_index([0])

In [10]:
labels

,1,2,3,4,5,6,7,8,9
0,,,,,,,,,
Postcode,Positional_quality_indicator,Eastings,Northings,Country_code,NHS_regional_HA_code,NHS_HA_code,Admin_county_code,Admin_district_code,Admin_ward_code


for file in files:
    postcode = pd.read_csv(r'{fname}'.format(fname=file), header=None)
    postcode = postcode.set_index([0])
    data = pd.concat([labels,postcode])
    data.columns = data.iloc[0]
    data = data.reindex(data.index.drop('Postcode'))
    SWW_postcode.merge(data, how='left', left_index=True, right_index=True)
    
    
    print('Processed {fname}'.format(fname=file))

In [21]:
postcode = pd.read_csv(r'/Users/annalisasheehan/Documents/Imperial-Diabetes/postcodes/codepo_gb/Data/CSV/ex.csv', header = None)

In [22]:
postcode

,0,1,2,3,4,5,6,7,8,9
0,EX1 1AE,10,291966,92583,E92000001,E19000002,E18000010,E10000008,E07000041,E05011020
1,EX1 1AF,10,292235,92324,E92000001,E19000002,E18000010,E10000008,E07000041,E05011020
2,EX1 1AJ,10,292170,92612,E92000001,E19000002,E18000010,E10000008,E07000041,E05011020
3,EX1 1AL,10,291978,92281,E92000001,E19000002,E18000010,E10000008,E07000041,E05011020
4,EX1 1AN,10,291989,92311,E92000001,E19000002,E18000010,E10000008,E07000041,E05011020
5,EX1 1AP,10,292291,92406,E92000001,E19000002,E18000010,E10000008,E07000041,E05011020
6,EX1 1AR,10,292310,92402,E92000001,E19000002,E18000010,E10000008,E07000041,E05011020
7,EX1 1AS,10,292383,92505,E92000001,E19000002,E18000010,E10000008,E07000041,E05011020
8,EX1 1AT,10,291781,92352,E92000001,E19000002,E18000010,E10000008,E07000041,E05011020
9,EX1 1AU,10,292401,92458,E92000001,E19000002,E18000010,E10000008,E07000041,E05011020


In [23]:
postcode = postcode.set_index([0])

In [24]:
data = pd.concat([labels,postcode])

In [25]:
data

,1,2,3,4,5,6,7,8,9
0,,,,,,,,,
Postcode,Positional_quality_indicator,Eastings,Northings,Country_code,NHS_regional_HA_code,NHS_HA_code,Admin_county_code,Admin_district_code,Admin_ward_code
EX1 1AE,10,291966,92583,E92000001,E19000002,E18000010,E10000008,E07000041,E05011020
EX1 1AF,10,292235,92324,E92000001,E19000002,E18000010,E10000008,E07000041,E05011020
EX1 1AJ,10,292170,92612,E92000001,E19000002,E18000010,E10000008,E07000041,E05011020
EX1 1AL,10,291978,92281,E92000001,E19000002,E18000010,E10000008,E07000041,E05011020
EX1 1AN,10,291989,92311,E92000001,E19000002,E18000010,E10000008,E07000041,E05011020
EX1 1AP,10,292291,92406,E92000001,E19000002,E18000010,E10000008,E07000041,E05011020
EX1 1AR,10,292310,92402,E92000001,E19000002,E18000010,E10000008,E07000041,E05011020
EX1 1AS,10,292383,92505,E92000001,E19000002,E18000010,E10000008,E07000041,E05011020


In [26]:
data.columns = data.iloc[0]

In [27]:
data = data.reindex(data.index.drop('Postcode'))

In [28]:
data

Postcode,Positional_quality_indicator,Eastings,Northings,Country_code,NHS_regional_HA_code,NHS_HA_code,Admin_county_code,Admin_district_code,Admin_ward_code
0,,,,,,,,,
EX1 1AE,10,291966,92583,E92000001,E19000002,E18000010,E10000008,E07000041,E05011020
EX1 1AF,10,292235,92324,E92000001,E19000002,E18000010,E10000008,E07000041,E05011020
EX1 1AJ,10,292170,92612,E92000001,E19000002,E18000010,E10000008,E07000041,E05011020
EX1 1AL,10,291978,92281,E92000001,E19000002,E18000010,E10000008,E07000041,E05011020
EX1 1AN,10,291989,92311,E92000001,E19000002,E18000010,E10000008,E07000041,E05011020
EX1 1AP,10,292291,92406,E92000001,E19000002,E18000010,E10000008,E07000041,E05011020
EX1 1AR,10,292310,92402,E92000001,E19000002,E18000010,E10000008,E07000041,E05011020
EX1 1AS,10,292383,92505,E92000001,E19000002,E18000010,E10000008,E07000041,E05011020
EX1 1AT,10,291781,92352,E92000001,E19000002,E18000010,E10000008,E07000041,E05011020


In [19]:
SWW_postcode = SWW_postcode.merge(data, how='left', left_index=True, right_index=True)

In [20]:
SWW_postcode

,date,Positional_quality_indicator,Eastings,Northings,Country_code,NHS_regional_HA_code,NHS_HA_code,Admin_county_code,Admin_district_code,Admin_ward_code
BH25 5LA,24/02/2017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
DT6,04/03/2003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
DT7 3AB,14/11/2008,10,333837,92488,E92000001,E19000002,E18000010,E10000009,E07000052,E05010594
DT7 3AD,05/05/2011,10,333914,92564,E92000001,E19000002,E18000010,E10000009,E07000052,E05010594
DT7 3AD,15/03/2010,10,333914,92564,E92000001,E19000002,E18000010,E10000009,E07000052,E05010594
DT7 3AD,02/09/2004,10,333914,92564,E92000001,E19000002,E18000010,E10000009,E07000052,E05010594
DT7 3BB,16/01/2013,10,333954,93132,E92000001,E19000002,E18000010,E10000009,E07000052,E05010594
DT7 3BB,07/03/2008,10,333954,93132,E92000001,E19000002,E18000010,E10000009,E07000052,E05010594
DT7 3BH,21/10/2011,10,334005,92854,E92000001,E19000002,E18000010,E10000009,E07000052,E05010594
DT7 3BJ,08/07/2016,10,333996,92661,E92000001,E19000002,E18000010,E10000009,E07000052,E05010594


In [31]:
data

Postcode,date,Positional_quality_indicator,Eastings,Northings,Country_code,NHS_regional_HA_code,NHS_HA_code,Admin_county_code,Admin_district_code,Admin_ward_code
0,,,,,,,,,,
EX1 1AE,NaT,10,291966,92583,E92000001,E19000002,E18000010,E10000008,E07000041,E05011020
EX1 1AF,NaT,10,292235,92324,E92000001,E19000002,E18000010,E10000008,E07000041,E05011020
EX1 1AJ,NaT,10,292170,92612,E92000001,E19000002,E18000010,E10000008,E07000041,E05011020
EX1 1AL,NaT,10,291978,92281,E92000001,E19000002,E18000010,E10000008,E07000041,E05011020
EX1 1AN,NaT,10,291989,92311,E92000001,E19000002,E18000010,E10000008,E07000041,E05011020
EX1 1AP,NaT,10,292291,92406,E92000001,E19000002,E18000010,E10000008,E07000041,E05011020
EX1 1AR,NaT,10,292310,92402,E92000001,E19000002,E18000010,E10000008,E07000041,E05011020
EX1 1AS,NaT,10,292383,92505,E92000001,E19000002,E18000010,E10000008,E07000041,E05011020
EX1 1AT,NaT,10,291781,92352,E92000001,E19000002,E18000010,E10000008,E07000041,E05011020


In [29]:
data.insert(0, 'date', 'NaN')

In [30]:
data['date']= pd.to_datetime(data['date'])

In [32]:
SWW_postcode[SWW_postcode.isnull()] = data

In [34]:
SWW_postcode.to_csv(r'/Users/annalisasheehan/Documents/Imperial-Diabetes/postcodes/test.csv')